In [1]:
import numpy as np 
import awkward as ak 
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
qcd = ak.firsts(ak.from_parquet('flat400.parquet'))

In [3]:
qcd = qcd[(qcd.goodjets.msoftdrop >= 80) & (qcd.goodjets.msoftdrop <= 170) & (qcd.goodjets.btag_count == 0)]

In [4]:
def nan_remover(sample):
    if type(sample) == dict:
        for j in sample:
            mask = ak.ones_like(sample[j][sample[j].fields[0]], dtype='bool')
            mask = ak.fill_none(mask, True)
            for k in sample[j].fields:
                mask = mask & (~ak.is_none(ak.nan_to_none(sample[j][k])))
            sample[j] = sample[j][mask]
    else:
        mask = ak.ones_like(sample[sample.fields[0]], dtype='bool')
        mask = ak.fill_none(mask, True)
        for j in sample.fields:
            if sample[j].fields == []:
                mask = mask & (~ak.is_none(ak.nan_to_none(sample[j])))
            else:
                for k in sample[j].fields:
                    mask = mask & (~ak.is_none(ak.nan_to_none(sample[j][k])))
        sample = sample[mask]
    return mask

In [5]:
len(qcd)

802234

In [6]:
mask = nan_remover(qcd.groomed_ecf_ratios) & nan_remover(qcd.ungroomed_ecf_ratios) & nan_remover(qcd.goodjets)

In [7]:
len(qcd[mask])

795006

In [8]:
sub = qcd[mask]

In [9]:
a = np.zeros((len(qcd),), dtype='bool')
a[::1] = True
arr = qcd[a]

In [10]:
len(qcd), len(arr), len(sub)

(802234, 802234, 795006)

In [11]:
ak.to_parquet(sub, 'nanless_flat400.parquet')

  created_by: parquet-cpp-arrow version 17.0.0
  num_columns: 339
  num_rows: 795006
  num_row_groups: 1
  format_version: 2.6
  serialized_size: 0

In [2]:
hgg = ak.from_parquet('powheg400_full_sample.parquet')

In [3]:
hgg = ak.firsts(hgg)

In [4]:
len(hgg)

1102488

In [5]:
hgg = hgg[(hgg.msoftdrop >= 80) & (hgg.msoftdrop <= 170)]

In [6]:
len(hgg)

797599

In [10]:
hgg = nan_remover(hgg)

In [11]:
len(hgg)

790840

In [12]:
ak.to_parquet(hgg, 'nanless_hgg_full_sample.parquet')

  created_by: parquet-cpp-arrow version 17.0.0
  num_columns: 165
  num_rows: 790840
  num_row_groups: 1
  format_version: 2.6
  serialized_size: 0